<a href="https://colab.research.google.com/github/mohammadbadi/Clustering_FE_MCA/blob/main/Code%20Sections/5.5%20KElbow%20Kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings                                                                   # Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from IPython.display import display, HTML
from google.colab import files

print("\n\n")
warnings.filterwarnings("ignore", category=DeprecationWarning)                    # Ignore Deprecation Warnings
warnings.filterwarnings("ignore", category=FutureWarning)                         # Ignore future warnings

url = "https://raw.githubusercontent.com/mohammadbadi/Clustering_FE_MCA/refs/heads/main/Output_CSV/FE_Encoded.csv"  # Read the dataset from CSV file
df = pd.read_csv(url)

continuous_features = ['OCC_YEAR', 'OCC_DOY', 'OCC_HOUR', 'LONG_WGS84', 'LAT_WGS84']

encoded_features = [
    # Cyclical encoding for month and day-of-week (and optionally hour)
    'OCC_MONTH_NUM_sin', 'OCC_MONTH_NUM_cos',
    'OCC_HOUR_sin', 'OCC_HOUR_cos',
    'OCC_DOW_NUM_sin', 'OCC_DOW_NUM_cos',

    # MCA components
    'LOCATION_TYPE_MCA_1', 'LOCATION_TYPE_MCA_2', 'LOCATION_TYPE_MCA_3', 'LOCATION_TYPE_MCA_4', 'LOCATION_TYPE_MCA_5',
    'HOOD_158_MCA_1',
    'DIVISION_MCA_1',

    # One-hot encoded features for PREMISES_TYPE
    'PREMISES_TYPE_Apartment', 'PREMISES_TYPE_Commercial', 'PREMISES_TYPE_Educational',
    'PREMISES_TYPE_House', 'PREMISES_TYPE_Other', 'PREMISES_TYPE_Outside', 'PREMISES_TYPE_Transit'
]
features = continuous_features + encoded_features

                                                                                  # Build preprocessor and pipeline for KMeans Clustering
scaler = StandardScaler()                                                         # Standardization
preprocessor = ColumnTransformer([('num', scaler, features)])
pipeline = Pipeline([                                                             # KMeans Clustering Pipeline (n_clusters=4 based on K-Elbow)
    ('preprocessor', preprocessor),
    ('kmeans', KMeans(n_clusters=4, random_state=42, n_init=10))
])
pipeline.fit(df)                                                                  # Fit the Pipeline
df_processed = pipeline.transform(df)                                             # Transform the Data

kmeans = pipeline.named_steps['kmeans']                                           # Extract KMeans Model, assign cluster labels, compute centroid variance
df['Cluster'] = kmeans.labels_
centroids = kmeans.cluster_centers_
centroid_variance = np.var(centroids, axis=0)
importance_df = pd.DataFrame({'Feature': features, 'Centroid Variance': centroid_variance})\
                  .sort_values(by='Centroid Variance', ascending=False)

sil_score = silhouette_score(df_processed, kmeans.labels_)                        # Compute Silhouette Score, Davies-Bouldin Index, Calinski-Harabasz Index, and Inertia
db_index = davies_bouldin_score(df_processed, kmeans.labels_)
ch_index = calinski_harabasz_score(df_processed, kmeans.labels_)
inertia_value = kmeans.inertia_

                                                                                  # ---------------------- Plot: K-Elbow Method ----------------------
inertia_values = []                                                               # Compute inertia for K values from 2 to 9
K_range = range(2, 10)
for k in K_range:
    kmeans_test = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_test.fit(df_processed)
    inertia_values.append(kmeans_test.inertia_)

plt.figure(figsize=(10, 5))
plt.plot(K_range, inertia_values, marker='o', linestyle='-')
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Inertia (Within-Cluster Sum of Squares)")
plt.title("Elbow Method for Optimal K")
plt.grid(True)
plt.savefig("5.5 K-Elbow.png", dpi=300, bbox_inches='tight')                      # Save Elbow plot
plt.show()
files.download("5.5 K-Elbow.png")                                                 # Download the Elbow plot
display(HTML("<br><br>"))

                                                                                  # ---------------------- Plot: K-Means Feature Importance ----------------------
plt.figure(figsize=(12, 6))
sns.barplot(data=importance_df, x='Centroid Variance', y='Feature', palette='viridis')
plt.xlabel("Centroid Variance")
plt.ylabel("Feature")
plt.title("Feature Importance in KMeans Clustering")
plt.grid(True, linestyle='--', alpha=0.6)
plt.savefig("5.6 K-Means Feature Importance.png", dpi=300, bbox_inches='tight')   # Save Feature Importance plot
plt.show()
files.download("5.6 K-Means Feature Importance.png")                              # Download Feature Importance plot
display(HTML("<br><br>"))

                                                                                  # ---------------------- Plot: Number of Points in Each Cluster ----------------------
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='Cluster', hue='Cluster', palette="Set2", dodge=False)
plt.title("Number of Points in Each Cluster")
plt.xlabel("Cluster Label")
plt.ylabel("Number of Data Points")
plt.grid(True, linestyle='--', alpha=0.6)
plt.savefig("5.6 K-Means Cluster Distribution.png", dpi=300, bbox_inches='tight') # Save Cluster Distribution plot
plt.show()
files.download("5.6 K-Means Cluster Distribution.png")                            # Download Cluster Distribution plot
display(HTML("<br><br>"))

                                                                                  # ---------------------- Build HTML Table for K-Means Clustering Analysis ----------------------
html_table = f"""
<table style='border-collapse: collapse; font-size: 18px; width: 100%; max-width: 900px; table-layout: fixed;'>
    <thead>
        <tr style='background-color: #2f4f4f; color: white;'>
            <th colspan="3" style="text-align: center; font-size: 24px; padding: 8px;">
                KMEANS Clustering Analysis
            </th>
        </tr>
        <tr>
            <td colspan="3" style="border: 1px solid #dddddd; padding: 8px; text-align: center; white-space: normal; word-wrap: break-word;">
                Based on the Optimal K value from the K-Elbow method, KMeans was performed with 4 clusters (n=4), which produced the following clustering statistics:
                <strong>Silhouette Score:</strong> {sil_score:.2f},
                <strong>Davies-Bouldin Index:</strong> {db_index:.2f},
                <strong>Calinski-Harabasz Index:</strong> {ch_index:.2f}, and
                <strong>Inertia:</strong> {inertia_value:.2f}.
                Centroid Variance represents the variance of the cluster centroids for each feature, reflecting its contribution to the clustering structure.
            </td>
        </tr>
        <tr style='background-color: #4CAF50; color: white;'>
            <th style='border: 1px solid #dddddd; padding: 8px;'>S/N</th>
            <th style='border: 1px solid #dddddd; padding: 8px;'>Feature</th>
            <th style='border: 1px solid #dddddd; padding: 8px;'>Centroid Variance</th>
        </tr>
    </thead>
    <tbody>
"""

for idx, row in enumerate(importance_df.itertuples(), start=1):                   # Add a row for each feature's centroid variance
    feature_val = row.Feature
    variance_val = f"{row[2]:.4f}"                                                # row[2] is the 'Centroid Variance'
    html_table += f"""
        <tr style='border: 1px solid #dddddd;'>
            <td style='border: 1px solid #dddddd; padding: 8px; text-align: center;'>{idx}</td>
            <td style='border: 1px solid #dddddd; padding: 8px;'>{feature_val}</td>
            <td style='border: 1px solid #dddddd; padding: 8px; text-align: right;'>{variance_val}</td>
        </tr>
    """
html_table += """
    </tbody>
</table>
"""

display(HTML(html_table))                                                         # Display the HTML table
display(HTML("<br><br>"))

                                                                                  # Save the HTML table as file "5.6 K-Means Feature Importance.html" and download it
html_table_path = "5.6 K-Means Feature Importance.html"
with open(html_table_path, "w", encoding="utf-8") as f:
    f.write(html_table)
files.download(html_table_path)                                                   # Download the HTML table file

df.to_csv("FE_Encoded_with_Clusters.csv", index=False)                            # Save the updated dataset with cluster labels
files.download("FE_Encoded_with_Clusters.csv")                                    # Download the updated dataset with cluster labels

                                                                                  # Display a message about the downloaded dataset
display(HTML("""
    <p style="color: darkblue; font-size: 18px; font-weight: bold;">
         Updated dataset with cluster labels saved as <span style="color: green; font-weight: bold;">FE_Encoded_with_Clusters.csv</span>.
    </p>
"""))